<a href="https://colab.research.google.com/github/soorajabraham1/my_projects/blob/main/Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Aug 31 10:23:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 6.4 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=c6a3305dc584dc01d9fc4b3a1c5f529dbfd4d8b0427e1d167c08a18ae1cfdb3c
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf


In [ ]:
data= yf.download("GOOGL" , start="2018-01-01" ,interval ='1d',end="2021-01-21")

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(768, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


In [ ]:
data.sort_index(inplace = True)

In [ ]:
data=data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-15,1729.020020,1747.619995,1711.709961,1727.619995,1727.619995,1572200
2021-01-19,1743.000000,1803.839966,1732.829956,1784.469971,1784.469971,2083000
2021-01-20,1826.369995,1899.719971,1819.510010,1880.069946,1880.069946,3207600


In [ ]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,768.000000,768.000000,768.000000,768.000000,768.000000,7.680000e+02
mean,1271.517005,1285.430675,1258.069194,1272.304882,1272.304882,1.851596e+06
std,202.058654,204.550711,200.868017,202.830141,202.830141,8.648732e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1118.329956,1129.585022,1106.924988,1117.007507,1117.007507,1.302325e+06
50%,1203.455017,1213.974976,1194.345032,1206.384949,1206.384949,1.634400e+06
75%,1416.522522,1434.617493,1394.937469,1413.505005,1413.505005,2.116100e+06
max,1826.369995,1899.719971,1819.510010,1880.069946,1880.069946,6.658900e+06


In [ ]:
import plotly.graph_objects as go
fig= go.Figure()
fig.add_trace(go.Scatter(x = data.index , y = data['Close'], mode='lines'))
fig.update_layout(height=500 , width=900,xaxis_title='Date' , yaxis_title='close')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [ ]:
data= data[['Close','Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


In [ ]:
#confirm test set length
test_length = data[(data.index >= '2020-09-01')].shape[0]
test_length

97

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
  X=[]
  Y=[]
  for i in tnrange(len(data) - feature_length):
    X.append(data.iloc[i : i + feature_length,:].values)
    Y.append(data["Close"].values[i+feature_length])

  X=np.array(X)
  Y=np.array(Y)
  return X,Y
  

In [ ]:
X,Y =CreateFeatures_and_Targets(data , 32)

  0%|          | 0/736 [00:00<?, ?it/s]

In [ ]:
X.shape,Y.shape

((736, 32, 2), (736,))

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = X[:-test_length] ,X[-test_length:] ,Y[:-test_length] , Y[-test_length:]

In [ ]:
test_length

97

In [ ]:
Xtrain.shape, Ytrain.shape

((639, 32, 2), (639,))

In [ ]:
Xtest.shape,Ytest.shape

((97, 32, 2), (97,))

In [ ]:
class MultiDimensionScaler():
    def __init__(self):
        self.scalers=[]

    def fit_transform(self , X):
        total_dims= X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:,:, i]=Scaler.fit_transform(X[:, :, i])   
            self.scalers.append(Scaler)
        return X

    def transform(self ,X):
        for i in range(X.shape[2]):
            X[:,:, i]=self.scalers[i].transform(X[:,:, i])
        return X

In [ ]:
X.shape[0]

736

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)
Xtrain

array([[[0.15972142, 0.15753971],
        [0.1927518 , 0.15381804],
        [0.19180902, 0.11007178],
        ...,
        [0.10534381, 0.15604439],
        [0.13348795, 0.23677477],
        [0.16178388, 0.20430983]],

       [[0.1927518 , 0.15381804],
        [0.20040051, 0.11007178],
        [0.2168967 , 0.14494584],
        ...,
        [0.13348795, 0.23677477],
        [0.16178388, 0.20430983],
        [0.16806177, 0.22551007]],

       [[0.20040051, 0.11007178],
        [0.22661192, 0.14494584],
        [0.22366487, 0.09837509],
        ...,
        [0.16178388, 0.20430983],
        [0.16806177, 0.22551007],
        [0.18032933, 0.17405463]],

       ...,

       [[0.96725832, 0.21022463],
        [0.96007865, 0.1513923 ],
        [0.91553443, 0.11366053],
        ...,
        [0.91056322, 0.10663255],
        [0.94195246, 0.10257859],
        [1.        , 0.32719147]],

       [[0.96007865, 0.1513923 ],
        [0.95654298, 0.11366053],
        [0.91886667, 0.17829135],
        .

In [ ]:
Ytrain.reshape(-1,1)

In [ ]:
Ytrain.shape

(639,)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain= Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest= Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name :str):
  pickle_out = open(f"{name}.pck","wb")
  pickle.dump(obj,pickle_out)
  pickle_out.close()

def load_object(name :str):
  pickle_in = open(f"{name}.pck","rb")
  data = pickle.load(pickle_in)
  return data

# **Model building**

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best= ModelCheckpoint("best_weights.h5" , moniter='val_loss' , save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5 , min_lr=0.00001, verbose=1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,Dropout,LSTM, Bidirectional

model=Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences =True , recurrent_dropout=0.1, input_shape=(32,2))))
model.add(LSTM(256,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='elu'))
model.add(Dense(1,activation='linear')) #final layer

In [ ]:
optimizer= tf.keras.optimizers.SGD(learning_rate=0.002)
model.compile(loss='mse',optimizer=optimizer)

In [ ]:
history= model.fit(Xtrain,Ytrain,
                   epochs=10,
                   batch_size=1,
                   verbose=1,
                   shuffle=False,
                   validation_data=(Xtest,Ytest),
                   callbacks=[reduce_lr,save_best])

In [ ]:
model.load_weights("best_weights.h5")

In [ ]:
Predictions=model.predict(Xtest)

In [ ]:
Predictions=Target_Scaler.inverse_transform(Predictions)
Actual= Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions.shape

(97, 1)

In [ ]:
Predictions=np.squeeze(Predictions ,axis=1)
Actual=np.squeeze(Actual,axis=1)



In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.index[-test_length:], y=Actual,mode='lines',name='Actual'))
fig.add_trace(go.Scatter(x=data.index[-test_length:], y=Predictions,mode='lines',name='Predicted'))

In [ ]:
Total_features= np.concatenate((Xtrain,Xtest) ,axis=0)

NameError: ignored

In [ ]:
Total_Targets= np.concatenate((Ytrain,Ytest) ,axis=0)

In [ ]:
predictions=model.predict(Total_features)

In [ ]:
Predictions=Target_Scaler.inverse_transform(Predictions)
Actual=Target_Scaler.inverse_transform(total_Targets)

In [ ]:
Predictions=np.squeeze(Predictions ,axis=1)
Actual=np.squeeze(Actual,axis=1)

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.index[-test_length:], y=Actual,mode='lines',name='Actual'))
fig.add_trace(go.Scatter(x=data.index[-test_length:], y=Predictions,mode='lines',name='Predicted'))

In [ ]:
import requests
response=requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM')
response=response.json()

In [ ]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [ ]:
rsi_data= pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index' )

In [ ]:
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [ ]:
rsi_data =rsi_data[rsi_data.index >= '2018-01-01']

In [ ]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [ ]:
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [ ]:
data=data.merge(rsi_data , left_index=True, right_index=True,how='inner')

In [ ]:
data.head()

,Close,Volume,RSI_x,RSI_y,RSI_x,RSI_y,RSI
2018-01-02,1073.209961,1588300,65.4310,65.4310,65.4310,65.4310,65.4310
2018-01-03,1091.520020,1565900,78.3686,78.3686,78.3686,78.3686,78.3686
2018-01-04,1095.760010,1302600,80.4830,80.4830,80.4830,80.4830,80.4830
2018-01-05,1110.290039,1512500,86.2429,86.2429,86.2429,86.2429,86.2429
2018-01-08,1114.209961,1232200,87.4882,87.4882,87.4882,87.4882,87.4882


Svae